In this notebook, we will use pre-trained deep learning model to process some text. We will then use the output of that model to classify the text. The text is a list of sentences from film reviews. And we will classify each sentence as either speaking "positively" about its subject or "negatively".

**Sentiment Analysis of Movie Review**

Our goal is to create a model that takes a sentence (just like the ones in our dataset) and produces either 1 (indicating the sentence carries a positive sentiment) or a 0 (indicating the sentence carries a negative sentiment). We can think of it as looking like this:

**Distil BERT**

DistilBERT is like a smaller, faster version of BERT. It reads a sentence and understands it. Then, it turns that understanding into a list of numbers (we call this a vector) that’s 768 items long.

Then, we take that vector and give it to a Logistic Regression model, which is a simple type of machine learning model. This model looks at the vector and decides whether the sentence is positive or negative.

So, in simple terms, DistilBERT reads and understands the sentence, and the Logistic Regression model makes the final decision.

**Datasets**


**SST2**, which contains sentences from movie reviews, each labeled as either positive (has the value 1) or negative (has the value 0):


In [1]:
!pip install transformers

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('/content/train.tsv', delimiter='\t', header=None)

In [4]:
len(df)

6920

In [5]:
batch_1 = df[:4000]

Loading The Pretrained BERT Model

In [6]:
# For DistilBERT:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Right now, the variable `model` holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

## Model #1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to do some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [7]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [8]:
len(tokenized[1])


16

In [9]:
# finding maximum length so that we can padd
max_len=0
for i in range(len(tokenized)):
  max_len=max(max_len,len(tokenized[i]))



In [10]:
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [11]:
np.array(padded).shape

(4000, 66)

### Masking
Now, the attention mask is created. This mask identifies which tokens are words (1) and which are padding (0).This tells the model to pay attention to the words and ignore the padding when processing the sentence.


In [12]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(4000, 66)

As Input is ready hence we can statrt Running Our Model

In [13]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [14]:
last_hidden_states[0].shape

torch.Size([4000, 66, 768])

We have to extract output for [CLS] .Why?
The [CLS] token, which stands for “classification”, is a special token that BERT adds to the beginning of each sentence. During pre-training, BERT learns to use this [CLS] token to represent the entire sentence, or more specifically, to represent the “classification task” for the sentence.

When BERT processes text data, it generates an output vector for each input token. So, for the [CLS] token, the corresponding output vector is effectively an embedding for the entire sentence, summarizing all its information. This is why we often use the output corresponding to the [CLS] token for sentence-level tasks.

In other words, the output of the [CLS] token can be thought of as a high-level representation of the whole sentence, capturing its overall semantic meaning. This makes it very useful for sentence-level classification tasks.

In [15]:
# extracting output for cls colmns for all sentences
features = last_hidden_states[0][:,0,:].numpy()

labels indicating which sentence is positive and negative now go into the labels variable



In [16]:
labels = batch_1[1]

Now We have features extracted By BERT and we have label also  now We can give this to any ml model . I am doing with Logistics Regression as it is simple

In [17]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [18]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression()

In [19]:
lr_clf.score(test_features, test_labels)

0.816

In [20]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier()
rf_clf.fit(train_features, train_labels)


RandomForestClassifier()

In [21]:
rf_clf.score(test_features, test_labels)

0.796

In [22]:
from sklearn import svm

svm_clf = svm.SVC()
svm_clf.fit(train_features, train_labels)


SVC()

In [23]:
svm_clf.score(test_features, test_labels)

0.836

In [24]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier()
xgb_clf.fit(train_features, train_labels)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [25]:
xgb_clf.score(test_features, test_labels)

0.805

In [26]:
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 5.263252631578947}
best scrores:  0.829
